In [28]:
import pandas as pd

## Wczytujemy dane
df = pd.read_csv(
    "../data/raw/Online_Retail.csv",
    encoding="ISO-8859-1"
)

## Sprawdzamy inital shape - ile wartości jest w datasecie
initial_shape = df.shape
initial_shape


(541909, 8)

In [3]:
## Sprawdzamy pierwsze 5 wartości
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/10 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/10 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/10 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/10 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/10 8:26,3.39,17850.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
## Podstawowe informacje o Data Secie: kolumny, ilew wartości non-null, jaki typ
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [6]:
## Sprawdzamy ile wartosci jest null w kazdej kolumnie
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [7]:
## Podstawowe informacje na temat Series Quantity
df["Quantity"].describe()

count    541909.000000
mean          9.552250
std         218.081158
min      -80995.000000
25%           1.000000
50%           3.000000
75%          10.000000
max       80995.000000
Name: Quantity, dtype: float64

In [8]:
df["UnitPrice"].describe()

count    541909.000000
mean          4.611114
std          96.759853
min      -11062.060000
25%           1.250000
50%           2.080000
75%           4.130000
max       38970.000000
Name: UnitPrice, dtype: float64

In [9]:
df["InvoiceDate"].head()

0    12/1/10 8:26
1    12/1/10 8:26
2    12/1/10 8:26
3    12/1/10 8:26
4    12/1/10 8:26
Name: InvoiceDate, dtype: object

In [10]:
## Sprawdzamy typ danych dla InvoiceDate
df["InvoiceDate"].dtype

dtype('O')

In [11]:
## Sprawdzwamy ile wartosci jest unikalnych w serii InvoiceNo
df["InvoiceNo"].nunique()

25900

In [12]:
df["InvoiceNo"].head()

0    536365
1    536365
2    536365
3    536365
4    536365
Name: InvoiceNo, dtype: object

In [13]:
# InvoiceNo nie jest unikalne dla każdego wiersza.
# Jedna faktura może zawierać wiele pozycji produktowych (line-item level).
#
# Konsekwencje:
# - InvoiceNo nie może być kluczem głównym
# - Agregacje wymagają groupby po InvoiceNo
# - Duża liczba wystąpień tej samej faktury jest poprawna
df["InvoiceNo"].value_counts().head()

InvoiceNo
573585    1114
581219     749
581492     731
580729     721
558475     705
Name: count, dtype: int64

In [14]:
# Checks if InvoiceNo contains any letters (regex); NaN treated as False
df["InvoiceNo"].str.contains("[A-Za-z]", na=False).sum()

np.int64(9291)

In [15]:
# Select InvoiceNo values containing letters and show first 10 unique values
df.loc[
    df["InvoiceNo"].str.contains("[A-Za-z]", na=False),
    "InvoiceNo"
].unique()[:10]

array(['C536379', 'C536383', 'C536391', 'C536506', 'C536543', 'C536548',
       'C536606', 'C536622', 'C536625', 'C536642'], dtype=object)

In [16]:
# Select Quantity where InvoiceNo starts with 'C', and give basic information about Quantity series
df.loc[
    df["InvoiceNo"].str.startswith("C", na=False),
    "Quantity"
].describe()

count     9288.000000
mean       -29.885228
std       1145.786965
min     -80995.000000
25%         -6.000000
50%         -2.000000
75%         -1.000000
max         -1.000000
Name: Quantity, dtype: float64

In [17]:
# Changes dataframe to remove (~df) InvoiceNo where it starts with 'C'
df = df[~df["InvoiceNo"].str.startswith("C", na=False)]
df.shape

(532621, 8)

In [18]:
# Change Quantity series - keep only values that are greater than 0
df = df[df["Quantity"] > 0]
df.shape

(531285, 8)

In [19]:
# Change UnitPrice series - keep only values that are greater than 0
df = df[df["UnitPrice"] > 0]
df.shape

(530104, 8)

In [20]:
# Change CustomerID series - leave only not null values
df = df[df["CustomerID"].notna()]
df.shape

(397884, 8)

In [21]:
df = df[df["Description"].notna()]
df.shape

(397884, 8)

In [22]:
# Remove unnecessary whitespaces and use upper case
df["Description"] = df["Description"].str.strip().str.upper()
df["Country"] = df["Country"].str.strip().str.upper()

df["Description"].tail()

541904        PACK OF 20 SPACEBOY NAPKINS
541905        CHILDREN'S APRON DOLLY GIRL
541906       CHILDRENS CUTLERY DOLLY GIRL
541907    CHILDRENS CUTLERY CIRCUS PARADE
541908       BAKING SET 9 PIECE RETROSPOT
Name: Description, dtype: object

In [23]:
# Change InvoiceDate series to DateTime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

# DT is an interface that helps with working on data (dt.year - returns year from DateTime)
df["InvoiceYear"] = df["InvoiceDate"].dt.year
df["InvoiceMonth"] = df["InvoiceDate"].dt.month

/var/folders/tb/q9083n095sx1tpb_1lrk6grr0000gn/T/ipykernel_42428/4001787485.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [24]:
df["TotalPrice"] = df["UnitPrice"] * df["Quantity"]

In [25]:
final_columns = [
    "InvoiceNo",
    "StockCode",
    "Description",
    "Quantity",
    "UnitPrice",
    "TotalPrice",
    "InvoiceDate",
    "InvoiceYear",
    "InvoiceMonth",
    "CustomerID",
    "Country"
]

df = df[final_columns]
df.shape

(397884, 11)

In [26]:
df.head()
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 397884 entries, 0 to 541908
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     397884 non-null  object        
 1   StockCode     397884 non-null  object        
 2   Description   397884 non-null  object        
 3   Quantity      397884 non-null  int64         
 4   UnitPrice     397884 non-null  float64       
 5   TotalPrice    397884 non-null  float64       
 6   InvoiceDate   397884 non-null  datetime64[ns]
 7   InvoiceYear   397884 non-null  int32         
 8   InvoiceMonth  397884 non-null  int32         
 9   CustomerID    397884 non-null  float64       
 10  Country       397884 non-null  object        
dtypes: datetime64[ns](1), float64(3), int32(2), int64(1), object(4)
memory usage: 33.4+ MB


,Quantity,UnitPrice,TotalPrice,InvoiceDate,InvoiceYear,InvoiceMonth,CustomerID
count,397884.000000,397884.000000,397884.000000,397884,397884.000000,397884.000000,397884.000000
mean,12.988238,3.116488,22.397000,2011-07-10 23:41:23.511023360,2010.934260,7.612475,15294.423453
min,1.000000,0.001000,0.001000,2010-12-01 08:26:00,2010.000000,1.000000,12346.000000
25%,2.000000,1.250000,4.680000,2011-04-07 11:12:00,2011.000000,5.000000,13969.000000
50%,6.000000,1.950000,11.800000,2011-07-31 14:39:00,2011.000000,8.000000,15159.000000
75%,12.000000,3.750000,19.800000,2011-10-20 14:33:00,2011.000000,11.000000,16795.000000
max,80995.000000,8142.750000,168469.600000,2011-12-09 12:50:00,2011.000000,12.000000,18287.000000
std,179.331775,22.097877,309.071041,NaN,0.247828,3.416520,1713.141560


In [29]:
from src.transform import transform_retail_data

df_clean = transform_retail_data(df)

df_clean.head()
df_clean.shape

/Users/patrykmaliszewski/PycharmProjects/Customer-Data-Project/src/transform.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  


(397884, 11)